In [1]:
from owlready2 import *

#we define our ontology; all ontologies share the same world by default
onto = get_ontology("http://test.org/onto.owl")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
with onto:
    #our entities are classes
    class Coffee(Thing): pass
    
    #related information can also be captured as classes
    class Roast(Thing): pass
    
    #subclassing Roast to break down additional details
    class Dark_Roast(Roast): pass
    class Blonde_Roast(Roast): pass
    class Medium_Roast(Roast): pass
    
    
    class Region(Thing): pass
    
    class Latin_America(Region): pass
    class Asia_Pacific(Region): pass
    class Multi(Region): pass
    
    
    #defining the relationship between coffee and roast
    class has_roast(ObjectProperty, FunctionalProperty):
        domain = [Coffee]
        region = [Roast]
    
    
    #FunctionalProperties mean it can only be related to one; these coffees can only be grown in one region
    class from_region(Coffee >> Region, FunctionalProperty):
        pass
    
    
    #defining the characteristics for a specific coffee type or line
    class Veranda(Coffee):
        equivalent_to = [Coffee & has_roast.value(Blonde_Roast) & from_region.some(Region) & 
                        from_region.only(Latin_America)]

    #.some means it must be related to a Region
    #.only means if it's related to a region it must be the one defined
    class Pike(Coffee):
        equivalent_to = [Coffee & has_roast.value(Medium_Roast) & from_region.some(Region) &
                        from_region.only(Latin_America)]

        

#telling the ontology these are all different things
AllDifferent([Dark_Roast, Blonde_Roast, Medium_Roast])
AllDifferent([Latin_America, Asia_Pacific, Multi])

#defining some unknown coffees and their characteristics
coffee1 = Coffee(has_roast = Blonde_Roast, from_region=Latin_America())
coffee2 = Coffee(has_roast = Medium_Roast, from_region=Latin_America())

print("Coffee 1 is: ", coffee1.is_a)
print("Coffee 2 is: ", coffee2.is_a)

#closing the world, by default we have an open world, anything not restricted is possible
close_world(Coffee)

Coffee 1 is:  [onto.Coffee]
Coffee 2 is:  [onto.Coffee]


In [3]:
#analyze the world, classify the coffees
sync_reasoner()

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /Users/paulbruffett/opt/anaconda3/envs/owl/lib/python3.9/site-packages/owlready2/hermit:/Users/paulbruffett/opt/anaconda3/envs/owl/lib/python3.9/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/mw/b9xyscgn6kq6b010hzq_73ym0000gn/T/tmplog2fl63
* Owlready2 * HermiT took 1.2318758964538574 seconds
* Owlready * Reparenting onto.coffee2: {onto.Coffee} => {onto.Pike}
* Owlready * Reparenting onto.coffee1: {onto.Coffee} => {onto.Veranda}
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [4]:
#what is coffee2 now classified as?
coffee1.is_a

[onto.Veranda]

In [5]:
coffee2.is_a

[onto.Pike]

In [6]:
#information about the parent class
Coffee.is_a

[owl.Thing,
 OneOf([onto.coffee1, onto.coffee2]),
 onto.has_roast.only(OneOf([onto.Blonde_Roast, onto.Medium_Roast, onto.Blonde_Roast, onto.Medium_Roast])),
 onto.from_region.only(onto.Region | onto.Region | OneOf([onto.latin_america1, onto.latin_america2]))]

In [7]:
Pike.is_a

[onto.Coffee,
 onto.has_roast.only(OneOf([onto.Medium_Roast])),
 onto.from_region.only(onto.Region)]